In [34]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.Collecting pandas
     --------------------------------------- 11.6/11.6 MB 31.2 MB/s eta 0:00:00
     ---------------------------------------- 347.8/347.8 KB ? eta 0:00:00
     --------------------------------------- 15.9/15.9 MB 41.0 MB/s eta 0:00:00
     ------------------------------------- 509.2/509.2 KB 33.3 MB/s eta 0:00:00



You should consider upgrading via the 'c:\ChatProject\chatproject_venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [35]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from kafka import KafkaProducer
from cassandra.cluster import Cluster
import json
import pandas as pd

In [14]:
producer = KafkaProducer(
    bootstrap_servers='127.0.0.1:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect('chat_system_data')

In [31]:
query = "SELECT * FROM servers;"  # Adjust to your Cassandra table name
results = session.execute(query)
servers = [{"server_id": row.server_id, "server_name": row.server_name} for row in results]  # Adjust fields as needed



In [32]:
print(servers)


[{'server_id': 1, 'server_name': 'testServer'}, {'server_id': 2, 'server_name': 'testServer2'}]


In [45]:
query = f"SELECT server_id FROM user_servers WHERE username='Bimby';"
result = session.execute(query)

query1 = f"SELECT * FROM servers;"
result1 = session.execute(query1)
rows1 = result1.all()
df1 = pd.DataFrame(rows1)
rows = result.all()
df = pd.DataFrame(rows)
print(df)

   server_id
0          2
1          3


In [46]:
print(df1)

   server_id  server_name
0          1   testServer
1          2  testServer2
2          3  testServer3


In [47]:
merged_df = df.merge(df1, on='server_id', how='inner')
print(merged_df)

   server_id  server_name
0          2  testServer2
1          3  testServer3


In [48]:
json_data = merged_df.to_dict(orient='records')
print(json_data)

[{'server_id': 2, 'server_name': 'testServer2'}, {'server_id': 3, 'server_name': 'testServer3'}]


In [53]:
query = f"SELECT * FROM messages_data WHERE server_id=3;"
result = session.execute(query)
rows = result.all()
df = pd.DataFrame(rows)
print(df)

   server_id username messages
0          3  battery     None


In [56]:
server_id = 3
query = f"SELECT * FROM messages_data WHERE server_id={server_id};"
result = session.execute(query) 
rows = result.all()
df = pd.DataFrame(rows)
dfSend = df.to_dict(orient='records')

print(dfSend)

[{'server_id': 3, 'username': 'battery', 'messages': None}]
